In [ ]:
import os
import re
import openai

In [ ]:
openai.api_key ="MY_API_KEY"

In [ ]:
# Calibrate - allows to understand if result is reasonable

prompt = "Context: You are a developer writing a detailed documentation for a function that allows the user to retrieve 'Last 15 Darkpool Trades utilizing the command `/dp alldp ticker`. How would you write a clear description for this command."

# The /dp alldp ticker command is a function that allows the user to retrieve the last 15 Darkpool Trades for a given ticker symbol. Darkpool trades are executed off-exchange, meaning they are not visible to the general public until after the trade has been completed. This command retrieves the most recent 15 trades for the given ticker symbol, providing the user with valuable information about the price and volume of trades executed off-exchange. This function can be used to gain insights into market sentiment and potentially inform trading strategies.

model_engine = "text-davinci-003"

response = openai.Completion.create(engine=model_engine, prompt=prompt, temperature=0.7, max_tokens=1024)

text = response.choices[0].text.strip()
print(text)

In [ ]:
def iterate_files_and_improve_documentation(folder_path):
    # Iterate through all files in the folder
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            # This allows to see if the location is correct
            print(file_path)

            # Check if the file is a markdown file
            if not filename.endswith(".md"):
                continue

            # Open the file in read mode
            with open(file_path, 'r+') as file:
                # Read the contents of the file
                file_contents = file.read()

                # Use regex to find our simple description
                cmd = filename.split(".")[0]

                start = f"# {cmd}\n"
                end = "### Usage"
            
                match = re.search(rf"{start}(.*?)\n{end}", file_contents, re.DOTALL)

                # If there is match, get description
                if match:
                    desc = match.group(1).replace("\n", "") # Remove newlines

                    # Use regex to find our example
                    start = "```"
                    end = "```"

                    matches = re.findall(rf"{start}(.*?){end}", file_contents, re.DOTALL)

                    # If there is match, get example
                    if matches:
                        # In our case the first match is a template, the second is the example
                        example = matches[1].replace("\n", "") # Remove newlines

                        # Use our prompt template for a more accurate description
                        prompt = f"Context: You are a developer writing a detailed documentation for a function that allows the user to retrieve {desc} utilizing the command {example} how would you explain what this command does in a single paragraph"

                        # Call OpenAI API
                        response = openai.Completion.create(engine=model_engine, prompt=prompt, temperature=0.7, max_tokens=1024)
                        if response:
                            improved_desc = response.choices[0].text.strip()

                            # Replace the old description with the improved one
                            file_contents = file_contents.replace(desc, improved_desc)

            # Open the file in write mode
            with open(file_path, 'w') as file:
                file.write(file_contents)
            
        elif os.path.isdir(file_path):
            # Recursively call this function on the subfolder
            iterate_files_and_improve_documentation(file_path)

# Call the function on the main folder
folder_path = "." # Current folder
iterate_files_and_improve_documentation(folder_path)